In [75]:
import sys
from pathlib import Path

# add project root to Python path
PROJECT_ROOT = Path.cwd().parents[0]
sys.path.append(str(PROJECT_ROOT))

print("Project root added to sys.path:", PROJECT_ROOT)

Project root added to sys.path: c:\Users\jinzh\Box\PR REPORT\11 GitHub\master-report


In [76]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from linearmodels.system import SUR
from statsmodels.stats.outliers_influence import variance_inflation_factor
from src.paths import RAW_DATA_DIR, PROCESSED_DATA_DIR, OUTPUTS_DIR, TABLES_DIR, FIGURES_DIR, SAMPLE_DATA_DIR

In [77]:
df17 = pd.read_csv(PROCESSED_DATA_DIR / "nhts2017_per_trip.csv")
df22 = pd.read_csv(PROCESSED_DATA_DIR / "nhts2022_per_trip.csv")

In [78]:
# Create sample csv 
df17.sample(2000, random_state=1).to_csv(SAMPLE_DATA_DIR/ "nhts2017_sample.csv")
df22.sample(2000, random_state=1).to_csv(SAMPLE_DATA_DIR/"nhts2022_sample.csv")

In [99]:
# By default, this notebook runs on sample data to allow execution
# without access to restricted NHTS microdata.
nhts2017_per_trip = pd.read_csv(SAMPLE_DATA_DIR / "nhts2017_sample.csv")
nhts2022_per_trip = pd.read_csv(SAMPLE_DATA_DIR / "nhts2022_sample.csv")

# To run the full analysis with complete NHTS datasets,
# comment out the lines above and uncomment the following:
# nhts2017_per_trip = pd.read_csv(PROCESSED_DATA_DIR / "nhts2017_per_trip.csv")
# nhts2022_per_trip = pd.read_csv(PROCESSED_DATA_DIR / "nhts2022_per_trip.csv")

In [80]:
nhts2017_per_trip = nhts2017_per_trip.rename(columns={"WKRMHM": "WRKFHM"})
nhts2022_per_trip = nhts2022_per_trip.rename(columns={"WKFMHM22": "WRKFHM"})
nhts2022_per_trip = nhts2022_per_trip.rename(columns={"RIDESHARE22": "RIDESHARE"})
nhts2017_per_trip["prepan"] = 1
nhts2022_per_trip["prepan"] = 0

In [81]:
nhts1722_per_trip = pd.concat([nhts2017_per_trip, nhts2022_per_trip], ignore_index=True)
nhts1722_per_trip

,Unnamed: 0,HOUSEID_x,PERSONID_x,R_AGE,R_HISP,R_SEX,R_RACE,WRKFHM,HHSIZE,HHVEHCNT,HHFAMINC,URBANSIZE,URBRUR,uni_Per_ID,YOUNGCHILD,Highschool,EDU_College,HM_OWN,NONHISWHITE,Race_Black,Race_Asian,Race_White,weekday,DRVRCNT,Total_TT,Total_AT,AT_ReguHMact,TT_ReguHMact,ReguHMact,AT_WorkFromHM,TT_WorkFromHM,WorkFromHM,AT_WorkNonHM,TT_WorkNonHM,WorkNonHM,AT_School,TT_School,School,AT_Shopping,TT_Shopping,Shopping,AT_Social1,TT_Social1,Social1,AT_Social2,TT_Social2,Social2,AT_Social3,TT_Social3,Social3,AT_Social4,TT_Social4,Social4,AT_Personal_self,TT_Personal_self,Personal_self,AT_Personal_meal,TT_Personal_meal,Personal_meal,AT_Family_business,TT_Family_business,Family_business,AT_Pick_Drop,TT_Pick_Drop,Pick_Drop,HW_TIME,ln_School,ln_Shopping,ln_Social1,ln_Social2,ln_Social3,ln_Social4,ln_Personal_self,ln_Personal_meal,ln_Family_business,ln_Pick_Drop,genZ,Millennials,genX,boomers2,prepan
0,33779,30395889,1,72.0,0.0,0.0,1.0,0,2,3,62.5,0,1,30395889_1,0,1,0,1,1,0,0.0,1.0,1,2,48,222,0,15,15,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,150,10,160,0,0,0,0,0,0,0,0,0,45,10,55,27,13,40,0,0,0,1185,0.00000,0.000000,0.00000,0.055004,0.000000,0.000000,0.0,0.019703,0.014418,0.000000,0,0,0,0,1
1,22273,30260300,4,20.0,1.0,1.0,97.0,0,6,3,30.0,0,1,30260300_4,0,1,0,1,0,0,0.0,0.0,1,3,20,15,0,5,5,0,0,0,0,0,0,0,0,0,15,15,30,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1410,0.00000,0.009143,0.00000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,1,0,0,0,1
2,81906,40671011,4,22.0,1.0,1.0,1.0,0,5,3,7.5,0,1,40671011_4,1,1,0,1,0,0,0.0,1.0,1,3,50,480,0,20,20,0,0,0,0,0,0,480,30,510,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,930,0.18988,0.000000,0.00000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0,1,0,0,1
3,9293,30108538,3,24.0,0.0,0.0,1.0,0,3,3,112.5,1,1,30108538_3,0,1,0,1,1,0,0.0,1.0,1,3,95,549,0,39,39,0,0,0,549,56,605,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,835,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0,1,0,0,1
4,39509,30464449,1,31.0,0.0,0.0,1.0,0,3,2,42.5,0,1,30464449_1,1,1,1,0,1,0,0.0,1.0,1,2,123,284,54,35,89,0,0,0,0,0,0,0,0,0,12,3,15,0,0,0,0,0,0,38,24,62,0,0,0,0,0,0,43,5,48,137,56,193,0,0,0,1122,0.00000,0.005768,0.00000,0.000000,0.023359,0.000000,0.0,0.018193,0.068933,0.000000,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,3306,9000147381,1,51.0,1.0,0.0,1.0,1,5,3,62.5,0,1,9000147381_1,0,1,0,1,0,0,0.0,1.0,1,4,90,255,0,0,0,0,45,45,255,45,300,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1140,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0,0,1,0,0
3996,1501,9000075591,1,48.0,0.0,1.0,1.0,0,7,5,112.5,0,1,9000075591_1,0,1,1,1,1,0,0.0,1.0,0,6,10,100,0,5,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,100,5,105,0,0,0,1335,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.0,0.000000,0.032881,0.000000,0,0,1,0,0
3997,4352,9000174755,1,30.0,0.0,0.0,1.0,0,4,1,30.0,1,1,9000174755_1,2,1,0,0,1,0,0.0,1.0,0,2,26,646,120,11,131,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,172,3,175,354,12,366,0,0,0,0,0,0,0,0,0,0,0,0,899,0.00000,0.000000,0.00000,0.000000,0.077245,0.148331,0.0,0.000000,0.000000,0.000000,0,1,0,0,0
3998,1024,9000055014,1,63.0,1.0,1.0,1.0,0,1,1,62.5,1,1,9000055014_1,0,1,1,1,0,0,0.0,1.0,0,1,100,335,20,30,50,0,0,0,180,15,195,0,0,0,15,15,30,85,15,100,15,15,30,0,0,0,0,0,0,0,0,0,20,10,30,0,0,0,0,0,0,1055,0.00000,0.012177,0.03933,0.012177,0.000000,0.000000,0.0,0.012177,0.000000,0.000000,0,0,0,1,0


In [82]:
nhts1722_per_trip = nhts1722_per_trip.dropna()

In [83]:
nhts1722_per_trip.to_csv("nhts1722_per_trip.csv", index=False)

In [84]:
nhts1722_per_trip

,Unnamed: 0,HOUSEID_x,PERSONID_x,R_AGE,R_HISP,R_SEX,R_RACE,WRKFHM,HHSIZE,HHVEHCNT,HHFAMINC,URBANSIZE,URBRUR,uni_Per_ID,YOUNGCHILD,Highschool,EDU_College,HM_OWN,NONHISWHITE,Race_Black,Race_Asian,Race_White,weekday,DRVRCNT,Total_TT,Total_AT,AT_ReguHMact,TT_ReguHMact,ReguHMact,AT_WorkFromHM,TT_WorkFromHM,WorkFromHM,AT_WorkNonHM,TT_WorkNonHM,WorkNonHM,AT_School,TT_School,School,AT_Shopping,TT_Shopping,Shopping,AT_Social1,TT_Social1,Social1,AT_Social2,TT_Social2,Social2,AT_Social3,TT_Social3,Social3,AT_Social4,TT_Social4,Social4,AT_Personal_self,TT_Personal_self,Personal_self,AT_Personal_meal,TT_Personal_meal,Personal_meal,AT_Family_business,TT_Family_business,Family_business,AT_Pick_Drop,TT_Pick_Drop,Pick_Drop,HW_TIME,ln_School,ln_Shopping,ln_Social1,ln_Social2,ln_Social3,ln_Social4,ln_Personal_self,ln_Personal_meal,ln_Family_business,ln_Pick_Drop,genZ,Millennials,genX,boomers2,prepan
0,33779,30395889,1,72.0,0.0,0.0,1.0,0,2,3,62.5,0,1,30395889_1,0,1,0,1,1,0,0.0,1.0,1,2,48,222,0,15,15,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,150,10,160,0,0,0,0,0,0,0,0,0,45,10,55,27,13,40,0,0,0,1185,0.00000,0.000000,0.00000,0.055004,0.000000,0.000000,0.0,0.019703,0.014418,0.000000,0,0,0,0,1
1,22273,30260300,4,20.0,1.0,1.0,97.0,0,6,3,30.0,0,1,30260300_4,0,1,0,1,0,0,0.0,0.0,1,3,20,15,0,5,5,0,0,0,0,0,0,0,0,0,15,15,30,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1410,0.00000,0.009143,0.00000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,1,0,0,0,1
2,81906,40671011,4,22.0,1.0,1.0,1.0,0,5,3,7.5,0,1,40671011_4,1,1,0,1,0,0,0.0,1.0,1,3,50,480,0,20,20,0,0,0,0,0,0,480,30,510,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,930,0.18988,0.000000,0.00000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0,1,0,0,1
3,9293,30108538,3,24.0,0.0,0.0,1.0,0,3,3,112.5,1,1,30108538_3,0,1,0,1,1,0,0.0,1.0,1,3,95,549,0,39,39,0,0,0,549,56,605,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,835,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0,1,0,0,1
4,39509,30464449,1,31.0,0.0,0.0,1.0,0,3,2,42.5,0,1,30464449_1,1,1,1,0,1,0,0.0,1.0,1,2,123,284,54,35,89,0,0,0,0,0,0,0,0,0,12,3,15,0,0,0,0,0,0,38,24,62,0,0,0,0,0,0,43,5,48,137,56,193,0,0,0,1122,0.00000,0.005768,0.00000,0.000000,0.023359,0.000000,0.0,0.018193,0.068933,0.000000,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,3306,9000147381,1,51.0,1.0,0.0,1.0,1,5,3,62.5,0,1,9000147381_1,0,1,0,1,0,0,0.0,1.0,1,4,90,255,0,0,0,0,45,45,255,45,300,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1140,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0,0,1,0,0
3996,1501,9000075591,1,48.0,0.0,1.0,1.0,0,7,5,112.5,0,1,9000075591_1,0,1,1,1,1,0,0.0,1.0,0,6,10,100,0,5,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,100,5,105,0,0,0,1335,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.0,0.000000,0.032881,0.000000,0,0,1,0,0
3997,4352,9000174755,1,30.0,0.0,0.0,1.0,0,4,1,30.0,1,1,9000174755_1,2,1,0,0,1,0,0.0,1.0,0,2,26,646,120,11,131,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,172,3,175,354,12,366,0,0,0,0,0,0,0,0,0,0,0,0,899,0.00000,0.000000,0.00000,0.000000,0.077245,0.148331,0.0,0.000000,0.000000,0.000000,0,1,0,0,0
3998,1024,9000055014,1,63.0,1.0,1.0,1.0,0,1,1,62.5,1,1,9000055014_1,0,1,1,1,0,0,0.0,1.0,0,1,100,335,20,30,50,0,0,0,180,15,195,0,0,0,15,15,30,85,15,100,15,15,30,0,0,0,0,0,0,0,0,0,20,10,30,0,0,0,0,0,0,1055,0.00000,0.012177,0.03933,0.012177,0.000000,0.000000,0.0,0.012177,0.000000,0.000000,0,0,0,1,0


In [85]:
ind_vars_1722 = [
    "R_AGE", "R_HISP", "R_SEX", "R_RACE", "HHSIZE", "HHVEHCNT", "HHFAMINC",
    "URBANSIZE", "URBRUR", "YOUNGCHILD", "Highschool", "EDU_College","WRKFHM",
    "HM_OWN", "NONHISWHITE", "Race_Black", "Race_White", "prepan","Race_Asian",
    "genZ", "Millennials", "genX", "boomers2",
     #added
    #"single_adult","multi_adult",
    #"driver",
    #"active_trans",  "small_motor", "transit", #"priv_veh",
    #"weekend", 
    "weekday",
    #"WRKCOUNT", "PTUSED","GCDWORK","NUMADLT","DELIVER",
    #"disability", 
    #"small_msa", "large_msa","has_rail",
    #"CNTTDTR","RIDESHARE",
    "DRVRCNT",
    #census
    #"northeast","midwest","south","west",
    #Household Income
    #"low_income","lower_mid_income", "middle_income", "upper_mid_income", "high_income",
    # urban size
    #"large_urban_rail","large_urban_norail","medium_urban","small_urban",
    
]
dep_vars_1722 = ["ln_School", "ln_Shopping", "ln_Social1", "ln_Social2", 
            "ln_Social3", "ln_Social4", "ln_Personal_self", 
            "ln_Personal_meal", "ln_Family_business", "ln_Pick_Drop"]

In [86]:
plt.figure(figsize=(24, 20))
X = nhts1722_per_trip[ind_vars_1722]
sns.heatmap(X.corr(), annot=True)
fig_path = FIGURES_DIR / "heatmap_ind_vars_1722.png"
plt.savefig(fig_path, dpi=200, bbox_inches="tight")
plt.close()
print("Saved figure to:", fig_path)

Saved figure to: C:\Users\jinzh\Box\PR REPORT\11 GitHub\master-report\outputs\figures\heatmap_ind_vars_1722.png


In [87]:
pd.set_option('display.max_columns', None)

In [88]:
nhts1722_per_trip[ind_vars_1722].describe()

,R_AGE,R_HISP,R_SEX,R_RACE,HHSIZE,HHVEHCNT,HHFAMINC,URBANSIZE,URBRUR,YOUNGCHILD,Highschool,EDU_College,WRKFHM,HM_OWN,NONHISWHITE,Race_Black,Race_White,prepan,Race_Asian,genZ,Millennials,genX,boomers2,weekday,DRVRCNT
count,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.00000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.00000,4000.000000
mean,45.509750,0.090750,0.482250,3.344000,2.750000,2.284750,109.268825,0.168000,0.800750,0.164000,0.96950,0.559250,0.292000,0.767000,0.771000,0.068500,0.832250,0.500000,0.054250,0.058000,0.318000,0.321000,0.214500,0.76775,2.054500
std,14.553981,0.287289,0.499747,13.908739,1.365271,1.169406,68.880674,0.373913,0.399486,0.482351,0.17198,0.496539,0.454739,0.422795,0.420242,0.252634,0.373691,0.500063,0.226539,0.233773,0.465758,0.466919,0.410526,0.42232,0.836484
min,16.000000,0.000000,0.000000,1.000000,1.000000,0.000000,7.500000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
25%,34.000000,0.000000,0.000000,1.000000,2.000000,2.000000,62.500000,0.000000,1.000000,0.000000,1.00000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.00000,2.000000
50%,46.000000,0.000000,0.000000,1.000000,2.000000,2.000000,87.500000,0.000000,1.000000,0.000000,1.00000,1.000000,0.000000,1.000000,1.000000,0.000000,1.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,1.00000,2.000000
75%,57.000000,0.000000,1.000000,1.000000,4.000000,3.000000,137.500000,0.000000,1.000000,0.000000,1.00000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.00000,2.000000
max,92.000000,1.000000,1.000000,97.000000,12.000000,12.000000,250.000000,1.000000,1.000000,4.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,9.000000


In [89]:
nhts1722_per_trip[dep_vars_1722].describe()

,ln_School,ln_Shopping,ln_Social1,ln_Social2,ln_Social3,ln_Social4,ln_Personal_self,ln_Personal_meal,ln_Family_business,ln_Pick_Drop
count,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000
mean,0.002577,0.007993,0.006390,0.004262,0.007739,0.002639,0.001657,0.006967,0.003067,0.003926
std,0.021967,0.017558,0.027227,0.016804,0.030505,0.014820,0.009922,0.018651,0.015981,0.017070
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.008686,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,0.357685,0.176091,0.342749,0.271217,0.385590,0.271872,0.204120,0.266268,0.364699,0.266389


In [90]:
#Check VIF
X = nhts1722_per_trip[ind_vars_1722].dropna()
vif_data = pd.DataFrame({
    "variable": X.columns,
    "VIF": [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
})
print(vif_data)

       variable        VIF
0         R_AGE  38.833800
1        R_HISP   7.111248
2         R_SEX   1.972257
3        R_RACE   2.672622
4        HHSIZE  12.784339
5      HHVEHCNT   9.658185
6      HHFAMINC   5.207757
7     URBANSIZE   1.360525
8        URBRUR   5.707036
9    YOUNGCHILD   1.691130
10   Highschool  34.412167
11  EDU_College   3.044636
12       WRKFHM   1.702259
13       HM_OWN   5.729184
14  NONHISWHITE  74.045371
15   Race_Black   4.060472
16   Race_White  86.899037
17       prepan   2.304261
18   Race_Asian   3.437545
19         genZ   3.287965
20  Millennials  10.127140
21         genX   6.226401
22     boomers2   3.353885
23      weekday   4.368098
24      DRVRCNT  20.323731


In [91]:
nhts1722_per_trip_final = nhts1722_per_trip[dep_vars_1722 + ind_vars_1722]

In [92]:
nhts1722_per_trip_final["const"] = 1

C:\Users\jinzh\AppData\Local\Temp\ipykernel_4340\1098599200.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nhts1722_per_trip_final["const"] = 1


In [93]:
ind_vars_1722_final = [
    "R_SEX",    
    #"HHSIZE",     
    "HHVEHCNT", 
    "HHFAMINC",
    "YOUNGCHILD", 
    "EDU_College",
    "HM_OWN", 

    "R_HISP",
    "Race_Black", 
    "Race_Asian",
    
    "prepan",
    "URBANSIZE", #rail or not
    "URBRUR", 
    "WRKFHM",
         
    #"multi_adult", #"single_adult",
    #"active_trans",  
    #"small_motor", 
    #"transit", 
    #"priv_veh",
    "weekday",
    #"PTUSED",
    #"DRVRCNT",
    #"GCDWORK",
    #"CNTTDTR",

    #GEN
    "genZ", "Millennials", 
    "genX", "boomers2",
    #"const"
]

In [94]:
#Check VIF
X = nhts1722_per_trip_final[ind_vars_1722_final].dropna()
vif_data = pd.DataFrame({
    "variable": X.columns,
    "VIF": [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
})
print(vif_data)

       variable       VIF
0         R_SEX  1.892001
1      HHVEHCNT  5.750105
2      HHFAMINC  5.079845
3    YOUNGCHILD  1.257474
4   EDU_College  2.951966
5        HM_OWN  5.049794
6        R_HISP  1.158258
7    Race_Black  1.115089
8    Race_Asian  1.099969
9        prepan  2.169966
10    URBANSIZE  1.358223
11       URBRUR  4.899098
12       WRKFHM  1.673918
13      weekday  3.916504
14         genZ  1.515215
15  Millennials  3.594440
16         genX  3.589788
17     boomers2  2.682286


In [95]:
# Build a dictionary of equations: each dependent variable is modeled as a function of the independent variables.
equations = {}
for dep in dep_vars_1722:
    
    formula = f"{dep} ~  " + " + ".join(ind_vars_1722_final)
    equations[dep] = formula

# check equations
for key, eq in equations.items():
     print(key, ":", eq)

model = SUR.from_formula(equations, data=nhts1722_per_trip_final)
model_sur = model.fit(iterate=True, cov_type='unadjusted')

print(model_sur.summary)
print("Asymptotic Variance-Covariance Matrix of Equations")
print(model_sur.resids.cov())

ln_School : ln_School ~  R_SEX + HHVEHCNT + HHFAMINC + YOUNGCHILD + EDU_College + HM_OWN + R_HISP + Race_Black + Race_Asian + prepan + URBANSIZE + URBRUR + WRKFHM + weekday + genZ + Millennials + genX + boomers2
ln_Shopping : ln_Shopping ~  R_SEX + HHVEHCNT + HHFAMINC + YOUNGCHILD + EDU_College + HM_OWN + R_HISP + Race_Black + Race_Asian + prepan + URBANSIZE + URBRUR + WRKFHM + weekday + genZ + Millennials + genX + boomers2
ln_Social1 : ln_Social1 ~  R_SEX + HHVEHCNT + HHFAMINC + YOUNGCHILD + EDU_College + HM_OWN + R_HISP + Race_Black + Race_Asian + prepan + URBANSIZE + URBRUR + WRKFHM + weekday + genZ + Millennials + genX + boomers2
ln_Social2 : ln_Social2 ~  R_SEX + HHVEHCNT + HHFAMINC + YOUNGCHILD + EDU_College + HM_OWN + R_HISP + Race_Black + Race_Asian + prepan + URBANSIZE + URBRUR + WRKFHM + weekday + genZ + Millennials + genX + boomers2
ln_Social3 : ln_Social3 ~  R_SEX + HHVEHCNT + HHFAMINC + YOUNGCHILD + EDU_College + HM_OWN + R_HISP + Race_Black + Race_Asian + prepan + URBANSI

In [97]:
from sklearn.metrics import r2_score

# Calculate R² for each dependent variable
r2_dict = {}
for dep_var in dep_vars_1722:
    actual = nhts1722_per_trip[dep_var]
    fitted = model_sur.fitted_values[dep_var]
    r2 = r2_score(actual, fitted)
    r2_dict[dep_var] = r2

# Display individual R² values
for dep_var, r2 in r2_dict.items():
    print(f"R² for {dep_var}: {r2:.4f}")



R² for ln_School: 0.0932
R² for ln_Shopping: 0.0331
R² for ln_Social1: 0.0193
R² for ln_Social2: 0.0152
R² for ln_Social3: 0.0256
R² for ln_Social4: 0.0234
R² for ln_Personal_self: 0.0132
R² for ln_Personal_meal: 0.0167
R² for ln_Family_business: 0.0038
R² for ln_Pick_Drop: 0.0195


In [102]:
with open(TABLES_DIR / "sur_1722_final.txt", "w") as f:
    f.write(str(model_sur.summary))
